# 前処理

## インポート

In [ ]:
from fabric import Connection, Config

## 接続情報
IPアドレスはDefault Switchで割り当てられたDHCPアドレスを取得

In [ ]:
remote_host = '192.168.128.10'
remote_user = 'vagrant'
key_file    = '.vagrant/machines/slurm-master/hyperv/private_key'

## fabric設定

In [ ]:
config = Config(overrides={
    'run': {'env': {'LANG': 'C'},'echo': True,'warn': True,'pty': False,'hide': False,'timeout': 20},
    'sudo': {'env': {'LANG': 'C'},'echo': True,'warn': True,'pty': True,'hide': False,'timeout': 20,'password': None},
    'timeouts': {'connect': 5,'command': 30,},
})

# 接続

## SSH接続

In [ ]:
c = Connection(
    port=22,
    host=remote_host,
    user=remote_user,
    connect_kwargs={
        'key_filename': key_file,
    },
    config=config,
)
c.run('hostname;date;whoami')

In [ ]:
c.run('mpicc --version')

In [ ]:
c.run('mpirun --version')

# 処理実行

In [ ]:
c.run("echo 'slurm-master slots=1\nslurm-node1 slots=1\nslurm-node2 slots=1\n' > ~/mpi-hosts")

In [ ]:
nodes = ["slurm-master", "slurm-node1", "slurm-node2"]

In [ ]:
for node in nodes:
    c.run(f"ssh -o StrictHostKeyChecking=no vagrant@{node} 'hostname'")

In [ ]:
hello_c = """#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(NULL, NULL);
    int world_size;
    MPI_Comm_size(MPI_COMM_WORLD, &world_size);
    int world_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);
    char processor_name[MPI_MAX_PROCESSOR_NAME];
    int name_len;
    MPI_Get_processor_name(processor_name, &name_len);
    printf("Hello from processor %s, rank %d out of %d processors\\n",
           processor_name, world_rank, world_size);
    MPI_Finalize();
    return 0;
}
"""

In [ ]:
for node in nodes:
    c.run(f"ssh vagrant@{node} 'cat > ~/hello.c <<EOF\n{hello_c}\nEOF'")

In [ ]:
escaped = hello_c.replace('\\', '\\\\').replace('"', '\\"').replace('$', '\\$')
for node in nodes:
    c.run(f"ssh vagrant@{node} 'echo \"{escaped}\" > ~/hello.c'")

In [ ]:
for node in nodes:
    c.run(f"ssh vagrant@{node} 'mpicc ~/hello.c -o ~/hello'")

In [ ]:
result = c.run("mpirun -np 3 -hostfile ~/mpi-hosts ~/hello", hide=True)
print(result.stdout)

In [ ]:
c.close()